In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [9]:
np.random.seed(42)
n_samples=1000

In [11]:
X_base = np.random.rand(n_samples, 1)
X = np.hstack([X_base + np.random.normal(0, 0.01, (n_samples, 1)) for _ in range(7)])
true_weights = np.array([3, 2.5, -1, 0.5, 4, -2, 1.5])
y = X.dot(true_weights) + np.random.normal(0, 0.1, n_samples)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features for stability
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
def ridge_regression_gd(X, y, alpha=0.01, lam=0.1, iterations=1000):
    n, m = X.shape
    X_b = np.c_[np.ones((n, 1)), X]
    theta = np.zeros((m + 1, 1))
    y = y.reshape(-1, 1)
    
    for i in range(iterations):
        gradients = (1/n) * X_b.T.dot(X_b.dot(theta) - y) + (lam/n) * np.r_[[[0]], theta[1:]]
        theta_new = theta - alpha * gradients
        
        # Check for numerical issues
        if np.any(np.isnan(theta_new)) or np.any(np.isinf(theta_new)):
            print(f"Warning: Divergence at iteration {i}, alpha={alpha}, lambda={lam}")
            return None
        
        theta = theta_new
    
    return theta

# Cost function
def ridge_cost(X, y, theta, lam):
    n = len(y)
    X_b = np.c_[np.ones((n, 1)), X]
    y_pred = X_b.dot(theta)
    mse = np.mean((y_pred - y.reshape(-1, 1)) ** 2)
    reg_term = (lam / (2*n)) * np.sum(theta[1:] ** 2)
    return mse + reg_term

In [13]:
learning_rates = [0.0001, 0.001, 0.01, 0.1]   # smaller range for stability
lambdas = [1e-10, 1e-5, 1e-3, 0, 1, 10]

results = []

for alpha in learning_rates:
    for lam in lambdas:
        theta = ridge_regression_gd(X_train, y_train, alpha=alpha, lam=lam, iterations=2000)
        if theta is None:
            continue  # skip invalid runs
        
        cost = ridge_cost(X_train, y_train, theta, lam)
        y_pred = np.c_[np.ones((X_test.shape[0], 1)), X_test].dot(theta)
        r2 = r2_score(y_test, y_pred)
        results.append((alpha, lam, cost, r2))

results_df = pd.DataFrame(results, columns=['Learning Rate', 'Lambda', 'Cost', 'R2_Score'])

# -----------------------------
# 4. Find best parameters
# -----------------------------
best_row = results_df.loc[results_df['R2_Score'].idxmax()]
print("\nBest Parameters:")
print(best_row)


Best Parameters:
Learning Rate    0.100000
Lambda           0.000000
Cost             0.012856
R2_Score         0.998228
Name: 21, dtype: float64


In [16]:
#q4
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [17]:
iris = load_iris()
X = iris.data
y = iris.target
n_classes = len(np.unique(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def add_bias(X):
    return np.c_[np.ones((X.shape[0], 1)), X]

# Binary Logistic Regression training using Gradient Descent
def train_logistic_regression(X, y, lr=0.1, epochs=2000):
    X_b = add_bias(X)
    m, n = X_b.shape
    theta = np.zeros((n, 1))
    y = y.reshape(-1, 1)

    for _ in range(epochs):
        z = X_b.dot(theta)
        h = sigmoid(z)
        gradient = (1/m) * X_b.T.dot(h - y)
        theta -= lr * gradient
    return theta

# Predict probability
def predict_proba(X, theta):
    X_b = add_bias(X)
    return sigmoid(X_b.dot(theta))

In [19]:
thetas = []
for c in range(n_classes):
    y_binary = (y_train == c).astype(int)
    theta_c = train_logistic_regression(X_train, y_binary, lr=0.1, epochs=3000)
    thetas.append(theta_c)

# Prediction: pick class with max probability
prob_matrix = np.hstack([predict_proba(X_test, theta) for theta in thetas])
y_pred = np.argmax(prob_matrix, axis=1)

# -----------------------------
# Evaluate performance
# -----------------------------
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:\n", cm)

Accuracy: 1.0000
Confusion Matrix:
 [[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
